# Plot the kinematics observables under isotropic inclination angle and lensing selected inclination angle

In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd
import corner
from copy import deepcopy
import glob

from cycler import cycler
import matplotlib as mpl
clist = ['#04B2D9', '#F2B33D', '#F29966', '#F2441D', '#0486C5']
mpl.rcParams['axes.prop_cycle'] = cycler(color=clist)

In [2]:
oblate = True

if oblate:
    oblate_name = 'oblate'
else:
    oblate_name = 'prolate'

In [3]:
def corner_plot_single_two_dataset(data_iso, data_sel, halo_id, qintr, plot_mean=True):

    color_rm = clist[0]
    color_sel = clist[2]

    hist2dkwargs = {'plot_density':False, 'plot_contours':False, 'plot_datapoints': True, 'color': color_rm, 'data_kwargs': {'ms': 5, 'alpha': 0.4}}
    labels = [r'$\theta_E$ [arcsec]', r'$e$', r'$\sigma^P$ [km/s]' ,r'$i$ [deg]']
    label_kwargs = {'fontsize': 22}
    hist_kwargs = {'linewidth': 2, 'density' : True}
    hist_kwargs_sel = {'linewidth': 2, 'density' : True}

    hist2dkwargs_sel = {'plot_density':False, 'plot_contours':False, 'plot_datapoints': True, 'color': color_sel, 'data_kwargs': {'ms': 10, 'alpha': 0.1}}

    fig = corner.corner(data_iso.T, labels=labels, label_kwargs=label_kwargs, **hist2dkwargs, hist_kwargs=hist_kwargs)
    corner.corner(data_sel.T, fig=fig, **hist2dkwargs_sel, hist_kwargs=hist_kwargs_sel)

    for ax in fig.get_axes():
        ax.tick_params(axis='both', labelsize=14)

    plt.plot([], [], color = color_rm, lw = 2, marker = ' ', label = 'isotropic inclination')
    plt.plot([], [], color = color_sel, lw = 2, marker = ' ', label = 'selected inclination')

    if plot_mean:
        mean_thetaE_rm = np.mean(data_iso[0, :])
        mean_thetaE_sel = np.mean(data_sel[0, :])
        mean_sigma_rm = np.mean(data_iso[2, :])
        mean_sigma_sel = np.mean(data_sel[2, :])

        ax_theta = fig.get_axes()[0]
        ax_theta.axvline(mean_thetaE_rm, ls = '--', marker = ' ', color = color_rm)
        ax_theta.axvline(mean_thetaE_sel, ls = '--', marker = ' ', color = color_sel)
        ax_theta.set_title(r'% diff = {:.3f} %' .format(np.abs(mean_thetaE_rm - mean_thetaE_sel) / np.mean([mean_thetaE_sel, mean_thetaE_rm]) * 100), fontsize = 16)

        ax_sigma = fig.get_axes()[10]
        ax_sigma.axvline(mean_sigma_rm, ls = '--', marker = ' ', color = color_rm)
        ax_sigma.axvline(mean_sigma_sel, ls = '--', marker = ' ', color = color_sel)
        ax_sigma.set_title(r'% diff = {:.3f} %' .format(np.abs(mean_sigma_rm - mean_sigma_sel) / np.mean([mean_sigma_sel, mean_sigma_rm]) * 100), fontsize = 16)

        plt.plot([], [], color = color_rm, ls = '--', marker = ' ', label = 'mean, isotropic')
        plt.plot([], [], color = color_sel, ls = '--', marker = ' ', label = 'mean, selected')

    fig.legend(bbox_to_anchor=(0., 0.95, 0.96 ,0), fontsize=21, frameon=False)
    # add title
    fig.suptitle('halo_id = {}, $q$ = {:.3f}' .format(halo_id, qintr), fontsize = 25, y = 1.05)
    plt.savefig('./corner_plot_{}/{}_halo_{}.pdf' .format(oblate_name, oblate_name, halo_id), dpi = 300, bbox_inches='tight')
    plt.close(fig)

In [4]:
# access catalog
cat = pd.read_pickle('../06-make_kin_mock_data/axisym_tng_catalog/{}_catalog.pkl' .format(oblate_name))

for i in range(len(cat)):

    gal_kwargs = cat.iloc[i].to_dict()
    halo_id = int(gal_kwargs.get('halo_id'))
    if oblate:
        qintr = gal_kwargs.get('xi_new') # modify if prolate
    else:
        qintr = 1/gal_kwargs.get('zeta_new')

    plot_flag = 1

    # find data
    try:
        file_isotropic = glob.glob('./data_{}/{}_isotropic_{}.npy' .format(oblate_name, oblate_name, halo_id))[0]
        data_iso = np.load(file_isotropic)
    except:
        plot_flag = -1
    try:
        file_selected = glob.glob('./data_{}/{}_selected_{}.npy' .format(oblate_name, oblate_name, halo_id))[0]
        data_sel = np.load(file_selected)
    except:
        plot_flag = -1

    if plot_flag != -1:
        corner_plot_single_two_dataset(data_iso, data_sel, halo_id, qintr, plot_mean=1)